In [1]:
def fit_gaussian_to_spot(image):
    """
    Implements 2D gaussian fitting to a noisy spot in the image, find the center (xc, yc) and 
    standard deviation (sigma) of the gaussian fit to the spot, all in pixel coordinates. 
    Perform the fit using unweighted least-squares fitting, in which the sum of squared errors in minimized.
    The first pixel (top-left corner) has coordinates (0,0). Background knowledge: the 
    image contains a single noisy gaussian spot with a width of approximately 5 pixels. 
    The intensity of the spot and number of background photons is arbitrary. 
    The pixel size is irrelevant, since everything is in pixel units.

    Input: 2D square grayscale image
    Output: Tuple: (xc, yc, sigma)
    """
    import numpy as np
    import scipy 

    image = np.asarray(image)
    num_px = image.shape[0]
    [xx, yy] = np.meshgrid(np.arange(0,num_px), np.arange(0,num_px))


    def twoDGauss(x,y,ux,uy,sigma):
        '''
        Creates theoretical normalized 2D gaussian spot, centered around (ux,uy) 
        with standard deviation sigma, on grid specified by x and y
        '''
        p = 1/(2*np.pi*sigma**2) * np.exp(- ( (x-ux)**2+(y-uy)**2 ) / (2*sigma**2))
        return p

    def expected(x,y,params):   
        '''
        Creates simulated realistic Gaussian spot on x-y grid, with following
        parameters = [x0,y0,sigma,bg,N], speficically:
            - x0 = x-position of gaussian (px coord., first pixel is (0,0))
            - y0 = y-positoin of gaussian (px coord.)
            - sigma = stdev of gaussian (px coord)
            - bg = number of photons in background
            - N = number of photons in spot
        '''
        E = params[4] * twoDGauss(x,y,params[0],params[1],params[2]) + params[3]
        return E

    def SSE(params):
        '''
        Calculates sum-of-squared errors difference between 'image' and expected Gaussian defined by 'params'
        This is essentially the cost-function that is minimized in the fit
        '''
        Expec = expected(xx,yy,params)
        sse = np.sum((image -  Expec)**2)
        return sse


    #guess fit parameters:
    x_guess = image.shape[0]/2      #assume gaussian is in the center of FOV
    x_guess = image.shape[0]/2      #assume gaussian is in the center of FOV
    sigma_guess = 2                 #assume sigma is 2 pixels
    bg_guess = (np.sum(image) - np.sum(image[1:-1,1:-1]))/(4*num_px-4)           
                                    #average of border pixels
    N_guess = np.sum(image) - bg_guess * num_px**2
                                    #sum of intensities, minus background
    params_guess = [x_guess,x_guess,sigma_guess, bg_guess, N_guess]

    #perform the fit
    param = scipy.optimize.fmin(func=SSE,x0=params_guess, maxiter=10000,maxfun=10000,ftol=1e-5,disp=0)

    xc_fit      = param[0]  #fitted x-position
    yc_fit      = param[1]  #fitted y-position
    sigma_fit   = param[2]  #fitted sigma

    return (xc_fit, yc_fit, sigma_fit)

In [2]:
def check(candidate):
    import numpy as np
    np.random.seed(11)  #fit random seed, so every initialization of noise is equal

    def twoDGauss(x,y,ux,uy,sigma):
        '''
        Creates theoretical normalized 2D gaussian spot, centered around (ux,uy) 
        with standard deviation sigma, on grid specified by x and y
        '''
        p = 1/(2*np.pi*sigma**2) * np.exp(- ( (x-ux)**2+(y-uy)**2 ) / (2*sigma**2))
        return p

    def expected(x,y,params):   
        '''
        Creates simulated realistic Gaussian spot on x-y grid, with following:
        parameters = [x0,y0,sigma,bg,N], with:
            - x0 = x-position of gaussian (px coord., first pixel is (0,0))
            - y0 = y-positoin of gaussian (px coord.)
            - sigma = stdev of gaussian (px coord)
            - bg = number of photons in background
            - N = number of photons in spot
        '''
        E = params[4] * twoDGauss(x,y,params[0],params[1],params[2]) + params[3]
        return E
    
    #Create sample image
    num_px = 10
    x_gt = 3.1      #x-position Gaussian
    y_gt = 4.9      #y-position Gaussian
    sigma_gt = 1.9  #sigma Gaussian
    N_gt = 1000     #number of signal photons
    bg_gt = 10      #number of background photons per pixel
    noise_gt = 10   #sigma of added noise
    param_gt = [x_gt,y_gt,sigma_gt,bg_gt,N_gt] 
    [xx, yy] = np.meshgrid(np.arange(0,num_px), np.arange(0,num_px))
    image = expected(xx,yy,param_gt) + np.random.randn(num_px,num_px)*noise_gt

    #perform the fit
    xc_fit, yc_fit, sigma_fit = candidate(image)
    
    #assess fitted parameters
    assert np.allclose(xc_fit,      x_gt,       0.1)
    assert np.allclose(yc_fit,      y_gt,       0.1)
    assert np.allclose(sigma_fit,   sigma_gt,   0.1)


In [3]:
check(fit_gaussian_to_spot)